# Лабораторная работа № 5
### Провести классификацию найденного датасета, методами решающего дерева и случайного леса. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.


## - Импорт библиотек

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

## - Инициализация датасета
#### (Датасет представляет собой данные о прошедших матчах в Бразильском CS:GO)

In [2]:
file_path = r'C:\Users\thebi\tb_lobby_stats_player.csv'

data = pd.read_csv(file_path)

num_rows, num_cols = data.shape
print(f"Количество строк: {num_rows}")
print(f"Количество столбцов: {num_cols}")


Количество строк: 184152
Количество столбцов: 38


In [3]:
print(data.columns)

Index(['idLobbyGame', 'idPlayer', 'idRoom', 'qtKill', 'qtAssist', 'qtDeath',
       'qtHs', 'qtBombeDefuse', 'qtBombePlant', 'qtTk', 'qtTkAssist',
       'qt1Kill', 'qt2Kill', 'qt3Kill', 'qt4Kill', 'qt5Kill', 'qtPlusKill',
       'qtFirstKill', 'vlDamage', 'qtHits', 'qtShots', 'qtLastAlive',
       'qtClutchWon', 'qtRoundsPlayed', 'descMapName', 'vlLevel', 'qtSurvived',
       'qtTrade', 'qtFlashAssist', 'qtHitHeadshot', 'qtHitChest',
       'qtHitStomach', 'qtHitLeftAtm', 'qtHitRightArm', 'qtHitLeftLeg',
       'qtHitRightLeg', 'flWinner', 'dtCreatedAt'],
      dtype='object')


## - Удаление столбцов с необрабатываемыми типами данных или просто не влияющих на исход матча

In [3]:
columns_to_drop = ['idRoom', 'idPlayer', 'idLobbyGame', 'qtFlashAssist', 'qtHitHeadshot', 'qtHitChest',
       'qtHitStomach', 'qtHitLeftAtm', 'qtHitRightArm', 'qtHitLeftLeg',
       'qtHitRightLeg', 'dtCreatedAt', 'descMapName']
data = data.drop(columns=columns_to_drop)

In [5]:
print(data.columns)

Index(['qtKill', 'qtAssist', 'qtDeath', 'qtHs', 'qtBombeDefuse',
       'qtBombePlant', 'qtTk', 'qtTkAssist', 'qt1Kill', 'qt2Kill', 'qt3Kill',
       'qt4Kill', 'qt5Kill', 'qtPlusKill', 'qtFirstKill', 'vlDamage', 'qtHits',
       'qtShots', 'qtLastAlive', 'qtClutchWon', 'qtRoundsPlayed', 'vlLevel',
       'qtSurvived', 'qtTrade', 'flWinner'],
      dtype='object')


## - Поиск пустых значений в датасете и удаление соответствующих строк

In [4]:
# Урезка датасета
data = data.sample(n=50000, random_state=42)

empty_columns = data.columns[data.isnull().any()]
empty_columns_count = data[empty_columns].isnull().sum()

# Вывод столбцов с пустыми значениями и их количество
if len(empty_columns) > 0:
    print("Столбцы с пустыми значениями и их количество:")
    for column in empty_columns:
        print(f"Столбец '{column}': {empty_columns_count[column]} пустых значений")
    print()

    # Удаление строк с пустыми значениями
    data_cleaned = data.dropna()
    print(f"Удалено строк с пустыми значениями: {len(data) - len(data_cleaned)}")
else:
    print("В датасете нет пустых значений")

Столбцы с пустыми значениями и их количество:
Столбец 'qtTk': 37 пустых значений
Столбец 'qtTkAssist': 37 пустых значений
Столбец 'qtHits': 37 пустых значений
Столбец 'qtLastAlive': 37 пустых значений
Столбец 'qtSurvived': 189 пустых значений
Столбец 'qtTrade': 189 пустых значений

Удалено строк с пустыми значениями: 189


## - Поиск столбцов с нечисловыми значениями

In [5]:
non_numeric_columns = data_cleaned.select_dtypes(exclude=['number']).columns.tolist()

if len(non_numeric_columns) > 0:
    print("Столбцы с нечисловыми значениями:")
    for column in non_numeric_columns:
        print(column)
else:
    print("Все столбцы содержат числовые значения")

# Подсчет количества строк и столбцов
num_rows, num_cols = data_cleaned.shape
print(f"Количество строк: {num_rows}")
print(f"Количество столбцов: {num_cols}")

Все столбцы содержат числовые значения
Количество строк: 49811
Количество столбцов: 25


## - Поиск лучших гиперпараметров

In [8]:
%%time
X = data_cleaned.drop('flWinner', axis=1)
y = data_cleaned['flWinner']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)


decision_tree = DecisionTreeClassifier()
dt_params = {
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

dt_grid_search = GridSearchCV(decision_tree, dt_params, cv=5)
dt_grid_search.fit(X_train, y_train)

y_pred_dt = dt_grid_search.predict(X_test)
accuracy_dt = accuracy_score(y_test, y_pred_dt)

print(f"Decision Tree - Лучшее значение параметра: {dt_grid_search.best_params_}, Точность на тесте: {accuracy_dt}")

random_forest = RandomForestClassifier()
rf_params = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_grid_search = GridSearchCV(random_forest, rf_params, cv=5)
rf_grid_search.fit(X_train, y_train)

y_pred_rf = rf_grid_search.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)

print(f"Random Forest - Лучшие значения параметров: {rf_grid_search.best_params_}, Точность на тесте: {accuracy_rf}")

Decision Tree - Лучшее значение параметра: {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 2}, Точность на тесте: 0.7684818067754078
Random Forest - Лучшие значения параметров: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 150}, Точность на тесте: 0.7841405269761607
Wall time: 35min 47s


## - Обучение модели 

In [9]:
%%time

best_dt_params = {'max_depth': 20}
best_decision_tree = DecisionTreeClassifier(**best_dt_params)
best_decision_tree.fit(X_train, y_train)

y_pred_best_dt = best_decision_tree.predict(X_test)
accuracy_best_dt = accuracy_score(y_test, y_pred_best_dt)

print(f"Decision Tree с лучшими параметрами - Точность на тесте: {accuracy_best_dt}")
print("Classification Report для Decision Tree:")
print(classification_report(y_test, y_pred_best_dt))

# Для случайного леса
best_rf_params = {'max_depth': None, 'n_estimators': 50}
best_random_forest = RandomForestClassifier(**best_rf_params)
best_random_forest.fit(X_train, y_train)

y_pred_best_rf = best_random_forest.predict(X_test)
accuracy_best_rf = accuracy_score(y_test, y_pred_best_rf)

print(f"Random Forest с лучшими параметрами - Точность на тесте: {accuracy_best_rf}")
print("Classification Report для Random Forest:")
print(classification_report(y_test, y_pred_best_rf))

Decision Tree с лучшими параметрами - Точность на тесте: 0.7283814303638645
Classification Report для Decision Tree:
              precision    recall  f1-score   support

           0       0.73      0.74      0.73     10083
           1       0.73      0.71      0.72      9842

    accuracy                           0.73     19925
   macro avg       0.73      0.73      0.73     19925
weighted avg       0.73      0.73      0.73     19925

Random Forest с лучшими параметрами - Точность на тесте: 0.7815307402760351
Classification Report для Random Forest:
              precision    recall  f1-score   support

           0       0.77      0.82      0.79     10083
           1       0.80      0.75      0.77      9842

    accuracy                           0.78     19925
   macro avg       0.78      0.78      0.78     19925
weighted avg       0.78      0.78      0.78     19925

Wall time: 2.63 s


__Из результатов лабораторной работы с моделями Decision Tree (Дерево решений) и Random Forest (Случайный лес) можно сделать следующие выводы:__

    Decision Tree:

Точность на тестовой выборке: 72.8%. Модель Decision Tree достигла точности классификации в 72.8% на тестовой выборке.
Classification Report:
Для класса 0 (первый класс) точность составляет 73%, полнота - 74%, F1-мера - 73%.
Для класса 1 (второй класс) точность равна 73%, полнота - 71%, F1-мера - 72%.
Общий вывод по Decision Tree: Модель достаточно хорошо справляется с классификацией обоих классов, хотя наблюдается небольшая разница между точностью и полнотой для обоих классов.

    Random Forest:
    
Точность на тестовой выборке: 78.2%. Random Forest показал более высокую точность классификации на тестовой выборке - 78.2%.
Classification Report:
Для класса 0 точность составляет 77%, полнота - 82%, F1-мера - 79%.
Для класса 1 точность равна 80%, полнота - 75%, F1-мера - 77%.

__Общий вывод по Random Forest: Эта модель демонстрирует более высокую точность классификации обоих классов, а также более сбалансированные значения точности и полноты по сравнению с Decision Tree.
В целом, Random Forest показал более высокую производительность по сравнению с Decision Tree на этом наборе данных, обеспечивая более высокую точность классификации и более сбалансированные метрики точности и полноты для обоих классов. Это свидетельствует о том, что Random Forest может быть предпочтительнее для решения данной задачи классификации.__